In [1]:
# from __future__ import print_function, unicode_literals, absolute_import, division
# import sys
import numpy as np
# import matplotlib
# matplotlib.rcParams["image.interpolation"] = 'none'
# import matplotlib.pyplot as plt
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
# from csbdeep.io import save_tiff_imagej_compatible

# from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
# from stardist.models import StarDist2D

np.random.seed(6)
# lbl_cmap = random_label_cmap()

In [2]:
#X = sorted(glob('data/dsb2018/test/images/*.tif'))
X = sorted(glob('/mnt/efs/aimbl_2025/student_data/S-LS/raw_bacteria/*.tif'))
X = list(map(imread,X))

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

In [3]:
from stardist.models import StarDist2D

ON_Stardist_model= StarDist2D(None, "Stardist_training_model", "/home/S-LS/AI-MBL-2025/masketeers/")

bioimageio_utils.py (2): pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
2025-09-03 21:03:34.341914: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-03 21:03:34.454945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-03 21:03:37.129317: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly di

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [ ]:
from skimage import io

outdir='/mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks.zarr'

masks_ext = ".png" if image_ext == ".png" else ".tif"
for i in len(X):
    img= X[i]
    norm_img = normalize(img, 1,99.8, axis=axis_norm)
    labels, details = ON_Stardist_model.predict_instances(norm_img)
    io.imsave('labels.tif', labels)
    io.imsave(outdir / (f.stem + "_masks" + masks_ext), labels)


# img = normalize(X, 1,99.8, axis=axis_norm)
# labels, details = ON_Stardist_model.predict_instances(img)

AttributeError: 'list' object has no attribute 'astype'

In [ ]:
import zarr

z= zarr.create_array(store="/mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks.zarr",
   shape=(2208, 2752),
   chunks=(552, 688),
   dtype="int32"
)

z[:, :] = labels

In [1]:
from glob import glob
import os
import numpy as np
from skimage.io import imread, imsave
from csbdeep.utils import normalize
from stardist.models import StarDist2D
#import zarr

# -------- gather inputs --------
X = sorted(glob('/mnt/efs/aimbl_2025/student_data/S-LS/raw_bacteria/*.tif'))
assert len(X) > 0, "No .tif images found."

# Peek first image to set sizes/axes
im0 = imread(X[0])
H, W = im0.shape[:2]
n_channel = 1 if im0.ndim == 2 else im0.shape[-1]
axis_norm = (0, 1)   # same as your snippet: normalize channels independently
if n_channel > 1:
    print("Normalizing image channels independently.")

# -------- load Stardist model (unchanged) --------
ON_Stardist_model = StarDist2D(None, "Stardist_training_model", "/home/S-LS/AI-MBL-2025/masketeers/")

# -------- outputs --------
tif_out_dir = '/mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif'
os.makedirs(tif_out_dir, exist_ok=True)

# zarr_path = "/mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks.zarr"
N = len(X)

# # Create Zarr stack with shape (N, H, W); chunk by slice for streaming writes
# z = zarr.open_array(
#     store=zarr_path,
#     mode="w",
#     shape=(N, H, W),
#     chunks=(552, 688),
#     dtype="int32"
# )

# -------- inference loop --------
for i, fpath in enumerate(X):
    img = imread(fpath)

    # # enforce common size so stack is consistent
    # if img.shape[:2] != (H, W):
    #     raise ValueError(f"Image size mismatch at {fpath}: got {img.shape[:2]}, expected {(H, W)}")

    norm_img = normalize(img, 1, 99.8, axis=axis_norm)

    # Stardist prediction: labels = 2D int mask
    labels, details = ON_Stardist_model.predict_instances(norm_img)
    labels = labels.astype(np.int32)

    # # save to Zarr stack
    # z[i, :, :] = labels

    # save individual TIFF outside the Zarr directory
    stem = os.path.splitext(os.path.basename(fpath))[0]
    imsave(os.path.join(tif_out_dir, f"{stem}_masks.tif"), labels)

#print(f"Done! Wrote {N} masks to {zarr_path} and saved TIFFs in {tif_out_dir}.")
print(f"Done! Wrote {N} masks to saved TIFFs in {tif_out_dir}.")

bioimageio_utils.py (2): pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
2025-09-04 00:04:50.376835: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-04 00:04:51.992367: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-04 00:04:57.359371: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly di

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_001_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_002_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_003_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_004_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_005_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_006_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_007_bacteria_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif/img_008_bacteria_masks.tif is a low contrast image
utils.py

Done! Wrote 72 masks to saved TIFFs in /mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks_tif.


In [1]:
from glob import glob
import os
import numpy as np
from skimage.io import imread, imsave
from csbdeep.utils import normalize
from stardist.models import StarDist2D
#import zarr

# -------- gather inputs --------
X = sorted(glob('/mnt/efs/aimbl_2025/student_data/S-LS/raw_bacteria/*.tif'))
assert len(X) > 0, "No .tif images found."

# Peek first image to set sizes/axes
im0 = imread(X[0])
H, W = im0.shape[:2]
n_channel = 1 if im0.ndim == 2 else im0.shape[-1]
axis_norm = (0, 1)   # same as your snippet: normalize channels independently
if n_channel > 1:
    print("Normalizing image channels independently.")

# -------- load Stardist model (unchanged) --------
first_model = StarDist2D.from_pretrained('2D_demo')
#ON_Stardist_model = StarDist2D(None, "Stardist_training_model", "/home/S-LS/AI-MBL-2025/masketeers/")

# -------- outputs --------
tif_out_dir = '/mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif'
os.makedirs(tif_out_dir, exist_ok=True)

# zarr_path = "/mnt/efs/aimbl_2025/student_data/S-LS/stardist_masks.zarr"
N = len(X)

# # Create Zarr stack with shape (N, H, W); chunk by slice for streaming writes
# z = zarr.open_array(
#     store=zarr_path,
#     mode="w",
#     shape=(N, H, W),
#     chunks=(552, 688),
#     dtype="int32"
# )

# -------- inference loop --------
for i, fpath in enumerate(X):
    img = imread(fpath)

    # # enforce common size so stack is consistent
    # if img.shape[:2] != (H, W):
    #     raise ValueError(f"Image size mismatch at {fpath}: got {img.shape[:2]}, expected {(H, W)}")

    norm_img = normalize(img, 1, 99.8, axis=axis_norm)

    # Stardist prediction: labels = 2D int mask
    labels, details = first_model.predict_instances(norm_img)
    labels = labels.astype(np.int32)

    # # save to Zarr stack
    # z[i, :, :] = labels

    # save individual TIFF outside the Zarr directory
    stem = os.path.splitext(os.path.basename(fpath))[0]
    imsave(os.path.join(tif_out_dir, f"{stem}_bt_masks.tif"), labels)

#print(f"Done! Wrote {N} masks to {zarr_path} and saved TIFFs in {tif_out_dir}.")
print(f"Done! Wrote {N} masks to saved TIFFs in {tif_out_dir}.")

bioimageio_utils.py (2): pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
2025-09-04 00:32:08.041386: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-04 00:32:08.150104: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-04 00:32:10.417985: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly di

Found model '2D_demo' for 'StarDist2D'.


W0000 00:00:1756945932.628706  664250 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_001_bacteria_bt_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_002_bacteria_bt_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_003_bacteria_bt_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_004_bacteria_bt_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_005_bacteria_bt_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_006_bacteria_bt_masks.tif is a low contrast image
utils.py (328): /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif/img_007_bacteria_bt_masks.tif is a low contrast image
utils.

Done! Wrote 72 masks to saved TIFFs in /mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif.


In [2]:
import os
from pathlib import Path
import glob
import numpy as np
import imageio
import zarr

# Inputs
bt_dir = "/mnt/efs/aimbl_2025/student_data/S-LS/beforetraining_stardist_masks_tif"
at_dir  = "/mnt/efs/aimbl_2025/student_data/S-LS/ON_stardist_masks_tif"

# Output Zarr store
store_path = "/mnt/efs/aimbl_2025/student_data/S-LS/stardist_output.zarr"
zarr.open(store_path, zarr_version=2)

# Open/create root and subgroups
root = zarr.group(store_path)
bt_grp = root.create_group("before_training") 
at_grp   = root.create_group("after_training")    

def stack_and_write(img_folder, group, dataset_name, chunks=(1,552,688)):
    files = sorted(glob.glob(os.path.join(img_folder, "*.tif")))
    N = len(files)
    # Load all images, stack to (N, H, W)
    stack = np.stack([imageio.imread(f) for f in files], axis=0)
    N, H, W = stack.shape
    

    ds = group.create_array(name=dataset_name, shape=(N, H, W), chunks=chunks, dtype="int32")
    ds[...] = stack

    ds.attrs["filenames"] = [Path(f).name for f in files]

    print(f" Wrote '{group.path}/{dataset_name}' | shape={(N,H,W)} | chunks={chunks} | dtype=int32")

# Build the two stacks
stack_and_write(bt_dir, bt_grp, "before_training_stack", chunks=(1,552,688))
stack_and_write(at_dir,  at_grp,   "after_training_stack",   chunks=(1,552,688))


print(root.tree())
print("hola")

2939204271.py (25): Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


 Wrote 'before_training/before_training_stack' | shape=(72, 2208, 2752) | chunks=(1, 552, 688) | dtype=int32
 Wrote 'after_training/after_training_stack' | shape=(72, 2208, 2752) | chunks=(1, 552, 688) | dtype=int32


/
├── after_training
│   └── after_training_stack (72, 2208, 2752) int32
└── before_training
    └── before_training_stack (72, 2208, 2752) int32


hola


In [1]:
import zarr
stardist_dir ='/mnt/efs/aimbl_2025/student_data/S-LS/stardist/stardist_output.zarr'
raw_dir = '/mnt/efs/aimbl_2025/student_data/S-LS/my_data2.zarr'

data_sd = zarr.open(stardist_dir, mode='r')
data_raw= zarr.open(raw_dir, mode='r')

before_training = data_sd['before_training']['before_training_stack'][:]
after_training = data_sd['after_training']['after_training_stack'][:]
raw_bacteria = data_raw['bacteria']['raw_bacteria_stack'][:]

In [2]:
import os
os.environ["DISPLAY"] = ':1001'

In [3]:
import napari
viewer=napari.Viewer()

In [4]:
viewer.add_image(raw_bacteria[32], name="raw_image")
viewer.add_labels(before_training[32], name='before')
viewer.add_labels(after_training[32], name='after')

/home/S-LS/conda/envs/stardist_env/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


<Labels layer 'after' at 0x78435dd18800>